# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [29]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt','wordnet'])

import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [35]:
a='abcd'
b=a[-3:-2]
print(b)

b


In [3]:
# load data from database
engine = create_engine('sqlite:///Cleandata.db')
df = pd.read_sql("SELECT * FROM Cleandata", engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = df.message
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [5]:
Y = df.iloc[:,4:40]
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)  # Punctuation Removal
    
    tokens=word_tokenize(text)   # tokenization
    
    lemmatizer=WordNetLemmatizer()  
    
    clean_tokens =[]
    
    for tok in tokens:
        
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        
        clean_tokens.append(clean_tok)
        
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('vect',  CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('mclf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [9]:
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
def display_results(Y_test, Y_pred):
    #labels = np.unique(Y_pred)
    #confusion_mat = confusion_matrix(Y_test, Y_pred, labels=labels)
    accuracy = (Y_pred == Y_test).mean()

    #print("Labels:", labels)
    ##print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)

In [11]:
Y_pred=pipeline.predict(X_test)

In [12]:
display_results(Y_test, Y_pred)

Accuracy: related                   0.794805
request                   0.873033
offer                     0.996028
aid_related               0.745760
medical_help              0.925745
medical_products          0.953705
search_and_rescue         0.972040
security                  0.981207
military                  0.970512
child_alone               1.000000
water                     0.945913
food                      0.921772
shelter                   0.928037
clothing                  0.985027
money                     0.978457
missing_people            0.989610
refugees                  0.969290
death                     0.963636
other_aid                 0.872574
infrastructure_related    0.937051
transport                 0.957372
buildings                 0.951261
electricity               0.981971
tools                     0.994194
hospitals                 0.988541
shops                     0.995722
aid_centers               0.987624
other_infrastructure      0.958136
weather_re

In [13]:
colname=list(Y_test.columns)
print(colname)

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [14]:
Y_test.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
21150,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12821,1,1,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,1
21033,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24525,1,0,0,1,0,1,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
10891,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
Y_pred

array([[1, 0, 0, ..., 0, 1, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [16]:
Y_pred2=pd.DataFrame(Y_pred, index=Y_pred[:,0])

In [17]:
Y_pred2.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,1,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [18]:
Y_test.iloc[:,0]

21150    1
12821    1
21033    1
24525    1
10891    1
15583    1
15560    1
24770    1
2900     0
7713     0
6425     1
21654    1
13670    1
8155     0
12954    1
8491     1
14277    0
24237    1
9017     1
12037    2
18344    1
19469    1
24122    0
14807    1
22406    1
14136    0
8628     0
5489     1
19359    1
3538     0
        ..
9815     1
5938     1
7556     1
17417    0
10279    1
16729    1
4320     1
4070     0
6878     0
24602    1
3015     0
20667    0
10343    1
21454    1
7656     0
21401    1
14611    1
23107    1
13569    1
5834     0
16904    1
10514    1
23873    1
16126    1
14677    1
13929    1
6914     1
7106     1
22111    0
12728    1
Name: related, Length: 6545, dtype: int64

In [19]:
Y_pred2.iloc[:,0]

1    1
1    1
1    1
1    1
1    1
1    1
1    1
1    1
1    1
0    0
0    0
1    1
1    1
0    0
1    1
1    1
1    1
1    1
1    1
2    2
1    1
0    0
1    1
1    1
1    1
1    1
0    0
1    1
1    1
1    1
    ..
0    0
0    0
1    1
1    1
1    1
0    0
1    1
1    1
1    1
1    1
1    1
1    1
1    1
1    1
0    0
1    1
1    1
1    1
1    1
0    0
1    1
1    1
1    1
1    1
1    1
1    1
1    1
1    1
1    1
1    1
Name: 0, Length: 6545, dtype: int64

In [20]:
target_names=['0','1']
for i in range(len(colname)):
    print(colname[i])
    print(classification_report(Y_test.iloc[:,i], Y_pred2.iloc[:,i], target_names=target_names))

related
             precision    recall  f1-score   support

          0       0.63      0.39      0.48      1551
          1       0.83      0.93      0.87      4949

avg / total       0.78      0.79      0.78      6545

request
             precision    recall  f1-score   support

          0       0.88      0.99      0.93      5407
          1       0.83      0.34      0.48      1138

avg / total       0.87      0.87      0.85      6545

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6519
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6545

aid_related
             precision    recall  f1-score   support

          0       0.74      0.87      0.80      3890
          1       0.75      0.56      0.64      2655

avg / total       0.75      0.75      0.74      6545

medical_help
             precision    recall  f1-score   support

          0       0.93      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 2
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 2
  .format(len(labels), len(target_names))


In [21]:
# modularize the classification_report
def classreport(Y_test, Y_pred):
    """
    modularize the classification_report
    
    """
    colname=list(Y_test.columns)
    
    Y_pred2=pd.DataFrame(Y_pred, index=Y_pred[:,0])
    
    target_names=['0','1']
    
    for i in range(len(colname)):
        print(colname[i])
        print(classification_report(Y_test.iloc[:,i], Y_pred2.iloc[:,i], target_names=target_names))
        
    return None

In [22]:
classreport(Y_test, Y_pred)

related
             precision    recall  f1-score   support

          0       0.63      0.39      0.48      1551
          1       0.83      0.93      0.87      4949

avg / total       0.78      0.79      0.78      6545

request
             precision    recall  f1-score   support

          0       0.88      0.99      0.93      5407
          1       0.83      0.34      0.48      1138

avg / total       0.87      0.87      0.85      6545

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6519
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6545

aid_related
             precision    recall  f1-score   support

          0       0.74      0.87      0.80      3890
          1       0.75      0.56      0.64      2655

avg / total       0.75      0.75      0.74      6545

medical_help
             precision    recall  f1-score   support

          0       0.93      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 2
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 2
  .format(len(labels), len(target_names))


### 6. Improve your model
Use grid search to find better parameters. 

In [23]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f2c5b737ae8>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('mclf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None

In [24]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'mclf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'mclf__estimator__bootstrap', 'mclf__estimator__class_weight', 'mclf__estimator__criterion', 'mclf__estimator__max_depth', 'mclf__estimator__max_features', 'mclf__estimator__max_leaf_nodes', 'mclf__estimator__min_impurity_decrease', 'mclf__estimator__min_impurity_split', 'mclf__estimator__min_samples_leaf', 'mclf__estimator__min_samples_split', 'mclf__estimator__min_weight_fraction_leaf', 'mclf__estimator__n_estimators', 'mclf__estimator__n_jobs', 'mclf__estimator__oob_score', 'mclf__estimator__random_state', 'mclf__estimator__verbose', 

In [25]:
parameters = {
    'mclf__estimator__n_estimators': [10, 50],
    'mclf__estimator__min_samples_split': [2, 3]
}

cv = GridSearchCV(pipeline, param_grid=parameters)


In [26]:
cv.fit(X_train, Y_train)
Y_pred3=cv.predict(X_test)

In [27]:
classreport(Y_test, Y_pred3)

related
             precision    recall  f1-score   support

          0       0.69      0.27      0.39      1551
          1       0.81      0.96      0.88      4949

avg / total       0.78      0.79      0.76      6545

request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5407
          1       0.85      0.42      0.56      1138

avg / total       0.88      0.89      0.87      6545

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6519
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6545

aid_related
             precision    recall  f1-score   support

          0       0.77      0.87      0.82      3890
          1       0.77      0.61      0.68      2655

avg / total       0.77      0.77      0.76      6545

medical_help
             precision    recall  f1-score   support

          0       0.93      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 2
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 2
  .format(len(labels), len(target_names))


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [31]:
pklfilename='picklefile.pkl'
gridcvpkl=open(pklfilename,'wb')
pickle.dump(cv, gridcvpkl)
gridcvpkl.close()

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.